该文件主要是为了生成需要的数据，并将数据保存到 data.db 中

In [18]:
import os
import string
import datetime
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [19]:
# 数据库地址：数据库放在上一级目录下
db_path = os.path.join(os.path.dirname(os.getcwd()),"data.db")
engine_path = "sqlite:///" + db_path
# 创建数据库引擎
engine = create_engine(engine_path)

### 商品退单率数据

In [31]:
shop_refuse_df = pd.DataFrame(
    data={
        "shopid":[n for e in [[i]*56 for i in range(1,81)] for n in e],
        "create_time":pd.date_range("2022-01-03", periods=56, freq="D").tolist()*80,
        "total_num":np.random.randint(100,300,80*56),
        "td_num":np.random.randint(10,30,80*56)
    }
)

In [35]:
shop_refuse_df.head()

,shopid,create_time,total_num,td_num
0,1,2022-01-03,173,19
1,1,2022-01-04,151,21
2,1,2022-01-05,173,23
3,1,2022-01-06,126,23
4,1,2022-01-07,148,28


In [36]:
shop_refuse_df.to_sql("shopRefuse", engine, if_exists='replace', index=True)

### 用户商品交易数据
> 适用于RFM模型
>
>user_id（用户id）、create_time（创建时间）、order_id（商品id）、amount（交易金额）

In [ ]:
sale_df = pd.read_csv("./sale.csv")

In [ ]:
sale_df.shape

In [ ]:
# 替换时间为 2020 年
sale_df["create_time"] = sale_df["create_time"].map(
    lambda x:x.replace("2018","2021") if pd.notnull(x) else x
)

In [ ]:
sale_df.sample(5)

In [ ]:
sale_df.to_sql("business", engine, if_exists='replace', index=True)

### 用户数据
>与comment表连用可以用于舆情分析
>
> 保存到数据库 users 表中
>
> user_id（用户id）、username（用户名）、age（年龄）
>

In [ ]:
name_str_list = [i for i in string.ascii_letters+string.digits]

In [ ]:
# 随机生成10000个用户的信息
user_df = pd.DataFrame(data={
    "user_id":range(1,10001),
    "username":["".join(np.random.choice(name_str_list,6)) for _ in range(10000)],
    "age":np.random.choice(range(20,50),10000),
})

In [ ]:
user_df.sample(2)

In [ ]:
user_df.to_sql("users", engine, if_exists='replace', index=False)

### 商品评价数据
> 商品评论数据来自 [阿里云天池比赛](https://tianchi.aliyun.com/competition/entrance/531890/introduction)
>
> 数据保存到数据库 comment 表中
>
>

In [ ]:
comment_df = pd.read_csv("./earphone_sentiment.csv")

In [ ]:
comment_df.sample(2)

In [ ]:
comment_df.shape

In [ ]:
# 添加创建时间
index = pd.date_range("2021-01-01", "2021-12-31")

# 随机选择 17176 条时间
comment_df["create_time"] = np.random.choice(a=index,size=17176)

# 添加用户id
comment_df["user_id"] = range(1,len(comment_df)+1)

In [ ]:
comment_df.head(2)

In [ ]:
comment_df.to_sql("comment", engine, if_exists='replace', index=False)

### 观研报告网数据
> 观研报告网：http://data.chinabaogao.com/ 
>
> read_html() 官网地址：[点击跳转](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html?highlight=read_html)
>
> 网页中有表格元素（tbody）均可以下载  


In [ ]:
# 举例说明，该 url 下有两个表格
# 其他页面数据，只需修改该 url 
url = "http://data.chinabaogao.com/dianxin/2021/11305620U2021.html"

result = pd.read_html(io=url,header=0)

for d in result:
    display(d)